In [1]:
import sys, os
sys.path.append(os.path.join(os.path.dirname('__file__'), '..', 'DB_and_Azure'))
import sql_db_functions as SQLf

### get data from server

In [2]:
conn, cursor = SQLf.sql_db_functions.connect_sql()

In [3]:
query = "SELECT product_characteristics.id, Brand_id , Detail, Summary, Brand FROM product_characteristics INNER JOIN Products ON product_characteristics.Brand_id = Products.Brand_Prod_id ;"
cursor.execute(query)

# Fetch the rows
rows = cursor.fetchall()
rows[0]

(161,
 1,
 '    *Type: Shirt\n    *Fit: Regular fit\n    *Sleeve style: Long sleeve\n    *Neckline: Collared\n    *Material: Silk\n    *Formality: Business Casual\n    *Season: Autumn, Spring, Summer\n    *Colors: Navy (70%), White (30%)\n    *Texture: Smooth\n    *Transparency: Opaque\n    *Details and Embellishments: Buttons\n    *Shape: Fitted\n    *Length: Hip-length\n    *Collar Style: Button-down\n    *Sleeve Style: Long\n    *Patterns: Geometric\n    *Patterns placement: All-over\n    *Fluidity of fabric: High\n    *Fabric weight: Light\n    *Pocket Presence: No\n    *Pocket placement: Not applicable\n    *Pocket size: Not applicable\n    *Breathability: High\n    *Occasion Suitability: Business Casual, Formal\n    *Lapel: Not applicable',
 'The item is a Gucci silk shirt featuring an optical GG monogram print. It is designed with a regular fit and features long sleeves with buttoned cuffs. The neckline is a classic pointed collar, adding a touch of formality. The material used 

In [4]:
conn, cursor = SQLf.sql_db_functions.connect_sql()

In [5]:
query = "SELECT Brand_id, base64 FROM product_img ;"
cursor.execute(query)

# Fetch the rows
rows_img = cursor.fetchall()
#rows_img

In [6]:
len(rows_img)

212

In [7]:
SQLf.sql_db_functions.close_connection_db(conn=conn,cursor=cursor)

### organize data

In [8]:
import pandas as pd

In [9]:
df = pd.DataFrame(rows ,columns= ['Id','prod_id','descripcion', 'Summary','Brand'])

In [10]:
df.head()

,Id,prod_id,descripcion,Summary,Brand
0,161,1,*Type: Shirt\n *Fit: Regular fit\n *...,The item is a Gucci silk shirt featuring an op...,Gucci
1,162,2,*Type: Shirt \n*Fit: Regular fit \n*Sleeve s...,The Gucci cotton Oxford GG shirt with an optic...,Gucci
2,163,3,*Type: Cropped blouse \n*Fit: Regular fit \n...,The Gucci top is a cropped blouse made of luxu...,Gucci
3,164,4,*Type: Shirt \n*Fit: Regular fit \n*Sleeve s...,The garment in question is a Gucci Camicia in ...,Gucci
4,165,5,*Type: Shirt \n*Fit: Regular fit \n*Sleeve s...,The garment in question is a Gucci cotton popl...,Gucci


In [11]:
df['descripcion'] = df['descripcion'].str.replace('\n*', ' / ')
df['descripcion'] = df['descripcion'].str.replace(r'\s+', ' ')
df['descripcion'] = df['descripcion'].str.replace('*', ' ')

df.head(5)

,Id,prod_id,descripcion,Summary,Brand
0,161,1,Type: Shirt\n Fit: Regular fit\n ...,The item is a Gucci silk shirt featuring an op...,Gucci
1,162,2,Type: Shirt / Fit: Regular fit / Sleeve s...,The Gucci cotton Oxford GG shirt with an optic...,Gucci
2,163,3,Type: Cropped blouse / Fit: Regular fit /...,The Gucci top is a cropped blouse made of luxu...,Gucci
3,164,4,Type: Shirt / Fit: Regular fit / Sleeve s...,The garment in question is a Gucci Camicia in ...,Gucci
4,165,5,Type: Shirt / Fit: Regular fit / Sleeve s...,The garment in question is a Gucci cotton popl...,Gucci


In [12]:
df['Complete_description'] = df['Summary'] + ' /' + "Brand: " + df['Brand'] + " /" + df['descripcion']
df.head()

,Id,prod_id,descripcion,Summary,Brand,Complete_description
0,161,1,Type: Shirt\n Fit: Regular fit\n ...,The item is a Gucci silk shirt featuring an op...,Gucci,The item is a Gucci silk shirt featuring an op...
1,162,2,Type: Shirt / Fit: Regular fit / Sleeve s...,The Gucci cotton Oxford GG shirt with an optic...,Gucci,The Gucci cotton Oxford GG shirt with an optic...
2,163,3,Type: Cropped blouse / Fit: Regular fit /...,The Gucci top is a cropped blouse made of luxu...,Gucci,The Gucci top is a cropped blouse made of luxu...
3,164,4,Type: Shirt / Fit: Regular fit / Sleeve s...,The garment in question is a Gucci Camicia in ...,Gucci,The garment in question is a Gucci Camicia in ...
4,165,5,Type: Shirt / Fit: Regular fit / Sleeve s...,The garment in question is a Gucci cotton popl...,Gucci,The garment in question is a Gucci cotton popl...


In [13]:
documents = df['Complete_description'].tolist()
id = df['Id'].astype('str').tolist()
metadata = [{"source": id} for id in df['Id'].astype('str').tolist()]

In [14]:
df_image = pd.DataFrame(rows_img, columns=['prod_id', 'base64'])

In [15]:
df_image.head(2)

,prod_id,base64
0,1,"data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ..."
1,1,"data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ..."


In [16]:
df_image.shape

(212, 2)

In [17]:
del rows, rows_img

### Get images 

In [18]:
import base64
from io import BytesIO
from PIL import Image

In [19]:
def base64_to_image(base64_string):
    # Remove the data URI prefix if present
    if "data:image" in base64_string:
        base64_string = base64_string.split(",")[1]

    # Decode the Base64 string into bytes
    image_bytes = base64.b64decode(base64_string)
    return image_bytes

In [20]:
def create_image_from_bytes(image_bytes):
    # Create a BytesIO object to handle the image data
    image_stream = BytesIO(image_bytes)

    # Open the image using Pillow (PIL)
    image = Image.open(image_stream)
    return image

In [21]:
def resize_image(image, new_width):
    # Open the image

    # Calculate the new height while maintaining the aspect ratio
    original_width, original_height = image.size
    aspect_ratio = original_height / original_width
    new_height = int(new_width * aspect_ratio)

    # Resize the image
    resized_image = image.resize((new_width, new_height), Image.ANTIALIAS)

    return resized_image


In [22]:
df_image.iloc[200].base64

'data:image/jpeg;base64, ...'

In [23]:
image_bytes = base64_to_image(df_image.iloc[42].base64)
create_image_from_bytes(image_bytes=image_bytes)

### Testing embeddings

In [24]:
import uuid
from langchain.vectorstores import Chroma
from langchain.storage import InMemoryStore
from langchain.schema.document import Document
from langchain.retrievers.multi_vector import MultiVectorRetriever

In [25]:
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

In [26]:
embedding_function = SentenceTransformerEmbeddings(model_name='all-mpnet-base-v2')

c:\Users\juand\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\juand\anaconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\Users\juand\anaconda3\Lib\site-packages\transformers\utils\generic.py:482: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\

### organized text 

In [27]:
df = pd.merge(
            df,
            df_image.groupby('prod_id')['base64'].apply(list).reset_index(),
            on='prod_id'
        )

In [28]:
df.prod_id = df.prod_id.astype(int)
df.sort_values('prod_id',ascending=True,inplace=True)
df.reset_index(drop=True, inplace= True)
df.prod_id = df.prod_id.astype('string')
df.head(5)

,Id,prod_id,descripcion,Summary,Brand,Complete_description,base64
0,161,1,Type: Shirt\n Fit: Regular fit\n ...,The item is a Gucci silk shirt featuring an op...,Gucci,The item is a Gucci silk shirt featuring an op...,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA..."
1,162,2,Type: Shirt / Fit: Regular fit / Sleeve s...,The Gucci cotton Oxford GG shirt with an optic...,Gucci,The Gucci cotton Oxford GG shirt with an optic...,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA..."
2,163,3,Type: Cropped blouse / Fit: Regular fit /...,The Gucci top is a cropped blouse made of luxu...,Gucci,The Gucci top is a cropped blouse made of luxu...,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA..."
3,164,4,Type: Shirt / Fit: Regular fit / Sleeve s...,The garment in question is a Gucci Camicia in ...,Gucci,The garment in question is a Gucci Camicia in ...,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA..."
4,165,5,Type: Shirt / Fit: Regular fit / Sleeve s...,The garment in question is a Gucci cotton popl...,Gucci,The garment in question is a Gucci cotton popl...,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA..."


### chunks

In [29]:
df = df[['prod_id', 'Summary','Brand','base64']]._append(df[['prod_id', 'descripcion','Brand','base64']].rename(columns = {'descripcion':'Summary'}),ignore_index=True)

In [30]:
df.Brand.unique()

array(['Gucci', 'HM', 'Mango', 'Prada'], dtype=object)

In [31]:
df = df[(df.Brand != 'Gucci') & (df.Brand != 'Prada')  ]
df.reset_index(inplace=True,drop=True)

In [32]:
df.head()

,prod_id,Summary,Brand,base64
0,14,"The shirt in the images is a contemporary, cas...",HM,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA..."
1,15,"The shirt featured in the images is a casual, ...",HM,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA..."
2,16,The shirt depicted in the images is a casual y...,HM,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA..."
3,17,The garment depicted in the images is a women'...,HM,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA..."
4,18,The garment is a casual women's long-sleeve sh...,HM,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA..."


In [33]:
persist_directory = '../../../Chromadb/chroma-retail'

In [34]:
store = InMemoryStore()
id_key = 'doc_id'

In [35]:
min(df['base64'].apply(lambda x: len(x)))

2

In [36]:
def return_none(x):

    if x == None: return 'None'
    
    return x

In [37]:
doc_ids = df.prod_id


texts = [
    Document(
        page_content= df.iloc[i].Summary, 
        metadata={
            id_key:df.iloc[i].prod_id,
            "img_1": return_none( df.iloc[i].base64[0] ),
            "img_2": return_none( df.iloc[i].base64[1] )  
            }
    )
    for i in df.index
]


In [38]:
vectorstore = Chroma.from_documents(
                documents=texts,
                #ids=doc_ids,
                embedding=embedding_function,
                collection_name='multi_modal_rag',
                #metadatas=metadata,
                persist_directory=persist_directory
            )

In [ ]:
vectorstore.similarity_search('HM')[0].page_content

### Already loaded !!!

In [ ]:
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
    search_kwargs={"k":5}
)

In [ ]:
docs = retriever.invoke(
    "HM"
)

In [ ]:
docs

In [ ]:
retriever.vectorstore.add_documents(texts)
#retriever.docstore.mset(list(zip(doc_ids, texts)))

In [ ]:
docs = retriever.invoke(
    "HM"
)

In [ ]:
docs

In [ ]:
df.iloc[1].Complete_description

### Talk to Chroma

In [ ]:
from apikey import apikey 
from langchain.chat_models import ChatOpenAI

In [ ]:
os.environ['OPENAI_API_KEY'] = apikey

In [ ]:
turbo_llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-3.5-turbo'
)

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

### lets set up the system

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
system_prompt = (
    "You are a Fashion specialist engine to sugest clothing pieces from context. "
    "If you don't know the answer to the question, say that you dont know "
    "answer concise."
    #"Explain why the selected options of the context are the best ones"
    "\n\n"
    "{context}"
)

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [ ]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | turbo_llm
    | StrOutputParser()
)

In [ ]:
question_answer_chain = create_stuff_documents_chain(turbo_llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
query = "Im looking for clothing peaces from Mango that could be similar to Gucci clothing"

In [ ]:
response = rag_chain.invoke({"input": query})
print(response["answer"])

In [ ]:
for document in response["context"]:
    print(document)
    print()

In [ ]:
df_image[df_image.prod_id == 41]

In [ ]:
image_bytes = base64_to_image(df_image.iloc[113].base64)
img = create_image_from_bytes(image_bytes=image_bytes)
img.resize((400,500))